In [1]:
%%time
import pandas as pd
import os, sys, time, json, re, string

from pyspark import SparkContext, SparkConf, StorageLevel, keyword_only

from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.ml.linalg import Vectors, VectorUDT
from pyspark.ml.param.shared import HasInputCol, HasInputCols, HasOutputCol, HasOutputCols, Param
from pyspark.ml.feature import HashingTF, IDF, Tokenizer, RegexTokenizer, NGram, CountVectorizer, StopWordsRemover
from pyspark.ml.feature import VectorAssembler, PCA

from pyspark.ml.classification import LogisticRegression, NaiveBayes, DecisionTreeClassifier, RandomForestClassifier, GBTClassifier, MultilayerPerceptronClassifier
from pyspark.ml import Pipeline, Transformer

from pyspark.ml.evaluation import MulticlassClassificationEvaluator, BinaryClassificationEvaluator
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
from platform import python_version
print(python_version())

3.6.10
CPU times: user 62 ms, sys: 4.2 ms, total: 66.2 ms
Wall time: 64.9 ms


In [2]:
%%time
sc

CPU times: user 3 µs, sys: 0 ns, total: 3 µs
Wall time: 6.44 µs


<SparkContext master=yarn appName=PySparkShell>

In [3]:
%%time
spark = SparkSession.builder \
        .appName("fakenews") \
        .config("spark.master", "yarn") \
        .config("spark.submit.deployMode", "cluster") \
        .config("spark.driver.memory", "25g") \
        .config("spark.executor.instances", "5") \
        .config("spark.executor.cores", "4") \
        .config("spark.executor.memory", "25g") \
        .getOrCreate()

CPU times: user 9.36 ms, sys: 0 ns, total: 9.36 ms
Wall time: 12 ms


In [10]:
%%time
fakenews_path="gs://dataproc-6ca41800-27b4-47d5-abee-55c011dfa389-asia-southeast1/data/fake-news/"
fakenews_data_path = fakenews_path + "two_million_rows_news_cleaned_2018_02_13_pyspark.csv"
df_news = spark.read.format("com.databricks.spark.csv") \
                    .option("header", "true") \
                    .option("delimiter", '#') \
                    .load(fakenews_data_path)

# only keep type and content
df_news = df_news.select("type", "content", "domain")
# add binary label
df_news = df_news.withColumn("label", F.when(F.col("type") == 'fake', 1).otherwise(0))

#remove empty content which will cause problem when transform the text
df_news = df_news.filter(df_news.content != "")

df_news_fake = df_news.filter(df_news.type == 'fake')
df_news_nonfake = df_news.filter(df_news.type != 'fake')
df_news = df_news_fake.union(df_news_nonfake)

# split the dataset
df_train, df_test = df_news.randomSplit([0.8, 0.2], seed=666)

param_tuning = False

CPU times: user 18.6 ms, sys: 4.28 ms, total: 22.9 ms
Wall time: 7.35 s


In [ ]:
%%time
# customized transformer class to manually extract some counting based text features
class ReviewContentTransformer(Transformer, HasInputCol, HasOutputCol):

    @keyword_only
    def __init__(self, inputCol="content", outputCol="content_features"):
        super(ReviewContentTransformer, self).__init__()
        kwargs = self._input_kwargs
        self.setParams(**kwargs)

    @keyword_only
    def setParams(self, inputCol=None, outputCol=None):
        kwargs = self._input_kwargs
        return self._set(**kwargs)


    def _transform(self, dataset):
        @udf('double')
        def f(s):
            uppercase_count = 0
            char_count = 0
            for c in s:                
                if c in string.ascii_uppercase:
                    uppercase_count += 1
                    char_count += 1
                elif c in string.ascii_lowercase:
                    char_count += 1
            
            text_len = len(s)
            return Vectors.dense(text_len, char_count, 
                                 uppercase_count, uppercase_count / (char_count + 1e-10))

        return dataset.withColumn(self.getOutputCol(), 
                                  F.udf(f, VectorUDT())(dataset[self.getInputCol()]))

In [ ]:
%%time
# customized transformer class to manually extract some counting based word features
class ReviewWordsTransformer(Transformer, HasInputCol, HasOutputCol):

    @keyword_only
    def __init__(self, inputCol="content", outputCol="content_features"):
        super(ReviewWordsTransformer, self).__init__()
        kwargs = self._input_kwargs
        self.setParams(**kwargs)

    @keyword_only
    def setParams(self, inputCol=None, outputCol=None):
        kwargs = self._input_kwargs
        return self._set(**kwargs)


    def _transform(self, dataset):
        
        def f(words):    
            word_count = len(words)
            unique_word_count = len(set(words))
            upper_words = []
            for w in words:
                if w.isupper():
                    upper_words.append(w)
            upper_word_count = len(set(upper_words))
            unique_upper_word_count = len(upper_words)
            return Vectors.dense(word_count, unique_word_count, unique_word_count / (word_count + 1e-10),
                                 upper_word_count, upper_word_count / (word_count + 1e-10), 
                                 unique_upper_word_count, unique_upper_word_count / (upper_word_count + 1e-10))

        return dataset.withColumn(self.getOutputCol(), 
                                  F.udf(f, VectorUDT())(dataset[self.getInputCol()]))

In [ ]:
%%time
# show model prediction performance on the given dataset
def eval_model_perf(fitted_model, dataset, label_col="label", prediction_col="prediction", probability_col="probability"):
    pred_dataset = fitted_model.transform(dataset)
    eval_dataset = pred_dataset.select(label_col, prediction_col, probability_col)
    # model performance evaluation
    metricNames = ["accuracy", "f1"]
    model_eval = MulticlassClassificationEvaluator(predictionCol=prediction_col, labelCol=label_col)
    for m in metricNames:
        val = model_eval.evaluate(eval_dataset, {model_eval.metricName: m})
        print(m, " = ", val)
    roc_eval = BinaryClassificationEvaluator(rawPredictionCol=probability_col, labelCol=label_col, metricName="areaUnderROC")
    print("AUC =", roc_eval.evaluate(eval_dataset))    
    return pred_dataset

# show CV param tunning result
def show_cv_results(cv_model):
    for result, param in sorted(zip(cv_model.avgMetrics, cv_model.getEstimatorParamMaps()), reverse=True, key=lambda x: x[0]):
        print(result, " | ", param)

In [ ]:
%%time
def run_models(df_train, df_test, without_pca = False):
    print("**********LogisticRegression**********")
    t = time.time()
    lr_model = LogisticRegression(featuresCol='features', 
                                  labelCol='label', 
                                  predictionCol='prediction', 
                                  probabilityCol='probability', 
                                  rawPredictionCol='rawPrediction',
                                  family='binomial', 
                                  fitIntercept=True, 
                                  threshold=0.5, 
                                  standardization=False, 
                                  maxIter=200, 
                                  regParam=0.005, 
                                  elasticNetParam=0, 
                                  tol=1e-06, 
                                  aggregationDepth=2)

    lr_model = lr_model.fit(df_train)
    
    eval_model_perf(lr_model, df_test)
    
    print("time taken for LogisticRegression: ", time.time() - t)
    t = time.time()

    print("**********DecisionTreeClassifier**********")
    dt_model = DecisionTreeClassifier(featuresCol='features', 
                                      labelCol='label', 
                                      predictionCol='prediction', 
                                      probabilityCol='probability', 
                                      rawPredictionCol='rawPrediction', 
                                      maxDepth=10, maxBins=32, 
                                      minInstancesPerNode=1, 
                                      minInfoGain=0.0, 
                                      maxMemoryInMB=2048, 
                                      cacheNodeIds=True, 
                                      checkpointInterval=10,
                                      impurity='gini', 
                                      seed=666)

    dt_model = dt_model.fit(df_train)
    eval_model_perf(dt_model, df_test)
    print("time taken for DecisionTreeClassifier: ", time.time() - t)
    t = time.time()
    
    print("**********RandomForestClassifier**********")
    rf_model = RandomForestClassifier(featuresCol='features', 
                                      labelCol='label', 
                                      predictionCol='prediction', 
                                      probabilityCol='probability', 
                                      rawPredictionCol='rawPrediction',
                                      maxDepth=10, 
                                      maxBins=32, 
                                      minInstancesPerNode=1, 
                                      minInfoGain=0.0, 
                                      maxMemoryInMB=2048, 
                                      cacheNodeIds=True, 
                                      checkpointInterval=10, 
                                      impurity='gini', 
                                      numTrees=200, 
                                      featureSubsetStrategy='auto', 
                                      seed=666, 
                                      subsamplingRate=0.8)

    rf_model = rf_model.fit(df_train)
    eval_model_perf(rf_model, df_test)
    print("time taken for RandomForestClassifier: ", time.time() - t)
    t = time.time()
    
    print("**********GBTClassifier**********")
    gbt_model = GBTClassifier(featuresCol='features', 
                             labelCol='label', 
                             maxIter=250)

    gbt_model = gbt_model.fit(df_train)
    eval_model_perf(gbt_model, df_test)
    print("time taken for GBTClassifier: ", time.time() - t)
    t = time.time()    
    
    print("**********MultilayerPerceptronClassifier**********")
    mp_model = MultilayerPerceptronClassifier(featuresCol='features', 
                                              labelCol='label', 
                                              predictionCol='prediction', 
                                              layers=[4, 5, 4, 3],  
                                              maxIter=100, 
                                              blockSize=128, 
                                              seed=1234)

    mp_model = mp_model.fit(df_train)
    eval_model_perf(mp_model, df_test)
    print("time taken for MultilayerPerceptronClassifier: ", time.time() - t)
    t = time.time() 
    
    if without_pca:
        print("**********NaiveBayes**********")
        nb_model = NaiveBayes(featuresCol='features', 
                              labelCol='label', 
                              predictionCol='prediction', 
                              probabilityCol='probability', 
                              rawPredictionCol='rawPrediction', 
                              smoothing=1, 
                              modelType='multinomial')

        nb_model = mp_model.fit(df_train)
        eval_model_perf(nb_model, df_test)
        print("time taken for NaiveBayes: ", time.time() - t)    

In [ ]:
%%time
def build_data_preproc_model_with_pca(vocab_size=5000):
    preproc_steps = [
        RegexTokenizer(inputCol="content", outputCol="all_words", pattern=r"\W"),
        StopWordsRemover(inputCol="all_words", outputCol="words"),
        CountVectorizer(inputCol="words", outputCol="tf_features", vocabSize=vocab_size),
        IDF(inputCol="tf_features", outputCol="tfidf_features"),
        PCA(inputCol="tfidf_features", outputCol="pca_features", k=100),
        ReviewContentTransformer(inputCol="content", outputCol="content_features"),
        ReviewWordsTransformer(inputCol="words", outputCol="word_features"),
        VectorAssembler(inputCols=["pca_features", "content_features", "word_features"], 
                        outputCol="features")
    ]
    return Pipeline(stages=preproc_steps)

def build_data_preproc_model_without_pca(vocab_size=5000):
    preproc_steps = [
        RegexTokenizer(inputCol="content", outputCol="all_words", pattern=r"\W"),
        StopWordsRemover(inputCol="all_words", outputCol="words"),
        CountVectorizer(inputCol="words", outputCol="tf_features", vocabSize=vocab_size),
        IDF(inputCol="tf_features", outputCol="tfidf_features"),
        ReviewContentTransformer(inputCol="content", outputCol="content_features"),
        ReviewWordsTransformer(inputCol="words", outputCol="word_features"),
        VectorAssembler(inputCols=["content_features", "word_features"], outputCol="features")
    ]
    return Pipeline(stages=preproc_steps)

In [ ]:
%%time
print("**********Run Models with PCA Features**********")
# generate the features to be used for model training
preproc_model = build_data_preproc_model_with_pca(2000).fit(df_train)
df_train_pca = preproc_model.transform(df_train).select("label", "features")
print(df_train_pca.take(1))
df_test_pca = preproc_model.transform(df_test).select("label", "features")
print(df_test_pca.take(1))
run_models(df_train_pca, df_test_pca)

In [ ]:
%%time
print("**********Run Models without PCA Features**********")
# generate the features to be used for model training
preproc_model = build_data_preproc_model_without_pca(3000).fit(df_train)
df_train_wo_pca = preproc_model.transform(df_train).select("label", "features")
print(df_train_wo_pca.take(1))
df_test_wo_pca = preproc_model.transform(df_test).select("label", "features")
print(df_test_wo_pca.take(1))
run_models(df_train_wo_pca, df_test_wo_pca, without_pca = True)